### Importações

In [10]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [11]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [12]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [13]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [14]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [15]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [16]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
6681,2023-11-14,Brasil Nbb,19:30,Pato,Bauru,2.69,1.45,151.5,1.88,1.79,4.5,1.94,1.77
6682,2023-11-14,Brasil Nbb,20:00,Corinthians Paulista,São Paulo,1.76,2.02,158.5,1.87,1.79,-2.5,2.01,1.72
6683,2023-11-14,Eua Nba,00:00,Sacramento Kings,Cleveland Cavaliers,2.01,1.93,225.5,1.86,2.07,1.5,1.88,2.07
6684,2023-11-14,Eua Nba,21:00,Charlotte Hornets,Miami Heat,2.21,1.56,226.5,1.87,1.79,1.5,2.03,1.66
6685,2023-11-14,Eua Nba,21:00,Detroit Pistons,Atlanta Hawks,2.78,1.45,232.5,1.88,1.96,5.5,1.90,1.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6775,2023-11-14,México Lnbp,23:00,Halcones de Xalapa,Fuerza Regia,2.81,1.42,158.5,1.82,1.92,4.5,1.99,1.75
6776,2023-11-14,México Lnbp,23:00,Plateros,Astros,2.18,1.66,176.5,1.86,1.88,1.5,2.02,1.71
6777,2023-11-14,México Lnbp,23:00,Santos,Abejas,2.03,1.75,167.5,1.80,1.93,1.5,1.89,1.85
6778,2023-11-14,México Lnbp,23:30,Panteras,Libertadores,1.77,2.01,171.5,1.87,1.87,-2.5,2.00,1.74


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,00:00,Eua Nba,Sacramento Kings,Cleveland Cavaliers,225.5,1.86,1.0000,Over
4,21:00,Eua Nba,Detroit Pistons,Atlanta Hawks,232.5,1.88,0.7044,Over


### Over v2 (NB)

In [18]:
# Duplicando o dataset
df_over_v2 = jogos.copy()

df_over_v2.reset_index(drop=True, inplace=True)
df_over_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v2.empty:
    mdl_over_v2 = load_model('ML/over/v2/v2_over', verbose=False)
    prev = predict_model(mdl_over_v2, df_over_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Last_CG_H'] >= 0) & (prev['Last_CG_H'] < 130)) | ((prev['Last_CG_H'] >= 183) & (prev['Last_CG_H'] < 1485))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V2 (NB) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V2 (NB) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,00:00,Eua Nba,Sacramento Kings,Cleveland Cavaliers,225.5,1.86,0.6696,Over
4,21:00,Eua Nba,Detroit Pistons,Atlanta Hawks,232.5,1.88,0.8192,Over
9,23:00,Eua Nba,Utah Jazz,Portland Trail Blazers,228.5,1.82,0.9424,Over
10,15:00,Europa Euroliga,Monaco,Anadolu Efes,162.5,1.91,0.7710,Over
12,16:30,Europa Euroliga,Valencia,Real Madrid,157.5,1.91,0.6933,Over
13,16:30,Europa Euroliga,Virtus Bologna,Milano,155.5,1.91,0.6943,Over
14,16:45,Europa Euroliga,Crvena zvezda,Fenerbahce,158.5,1.91,0.8235,Over
22,08:35,China Cba,Zhejiang Guangsha,Zhejiang Chouzhou,200.5,1.88,0.6861,Over
23,08:30,China Wcba Feminina,Shanxi Xing Rui F,Hebei F,151.5,1.80,0.9519,Over
26,07:00,Coreia Do Sul Kbl,Seoul Thunders,Anyang JungKwanJang,162.5,1.80,0.6113,Over


### Over v3 (NB)

In [19]:
# Duplicando o dataset
df_over_v3 = jogos.copy()

df_over_v3.reset_index(drop=True, inplace=True)
df_over_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v3.empty:
    mdl_over_v3 = load_model('ML/over/v3/v3_over', verbose=False)
    prev = predict_model(mdl_over_v3, df_over_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V3 (NB) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V3 (NB) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,00:00,Eua Nba,Sacramento Kings,Cleveland Cavaliers,225.5,1.86,0.7440,Over
4,21:00,Eua Nba,Detroit Pistons,Atlanta Hawks,232.5,1.88,0.8053,Over
9,23:00,Eua Nba,Utah Jazz,Portland Trail Blazers,228.5,1.82,0.5769,Over
10,15:00,Europa Euroliga,Monaco,Anadolu Efes,162.5,1.91,0.7920,Over
11,16:05,Europa Euroliga,Maccabi Tel Aviv,Olympiakos,158.5,1.87,0.7733,Over
12,16:30,Europa Euroliga,Valencia,Real Madrid,157.5,1.91,0.7993,Over
13,16:30,Europa Euroliga,Virtus Bologna,Milano,155.5,1.91,0.7891,Over
14,16:45,Europa Euroliga,Crvena zvezda,Fenerbahce,158.5,1.91,0.8092,Over
22,08:35,China Cba,Zhejiang Guangsha,Zhejiang Chouzhou,200.5,1.88,0.7575,Over
27,07:00,Coreia Do Sul Kbl,Wonju DB,KCC Egis,175.5,1.87,0.5368,Over
